In [2]:
import tiktoken
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn

In [3]:
DIR = "DATA/the-verdict.txt"
with open(DIR, "r", encoding="utf-8") as f:
    raw_data = f.read()
print(raw_data[:99])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [23]:
class DatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_len, stride):
        super().__init__()
        self.input_idx = []
        self.target_idx = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_len, stride):
            input_chunk = token_ids[i: i+max_len]
            target_chunk = token_ids[i+1:i+max_len+1]

            self.input_idx.append(torch.tensor(input_chunk))
            self.target_idx.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_idx)
    
    def __getitem__(self,idx):
        return self.input_idx[idx], self.target_idx[idx]

In [5]:
def crete_data_loader(txt, max_len, stride,
                      batch_size=1,shuffle=True,drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    data_set = DatasetV1(txt,
                         tokenizer,
                         max_len,
                         stride)
    dataloader = DataLoader(dataset=data_set,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            drop_last=drop_last,
                            num_workers=num_workers)
    return dataloader

In [6]:
vocab_size = 50257
embedding_dim = 256
token_embedding_layer = nn.Embedding(vocab_size, embedding_dim)

In [7]:
dataloader = crete_data_loader(txt=raw_data, max_len=4, stride=4, batch_size=8,shuffle=False)

In [8]:
data_itr = iter(dataloader)
data_batch_1 = next(data_itr)
inputs,targets = data_batch_1
print(f"INPUT:\n{inputs}\nOUTPUT:\n{targets}")

INPUT:
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
OUTPUT:
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [9]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings[0])

tensor([[-0.3288,  0.3586,  0.7055,  ..., -1.5553, -0.1219,  1.2151],
        [ 0.9036,  0.0782, -0.9076,  ..., -0.9605,  1.3519, -1.3649],
        [-0.5587, -0.6353, -2.0324,  ..., -2.3393, -1.3733,  0.2610],
        [-1.8043,  1.3405,  0.5743,  ..., -0.5202,  1.0497,  0.1272]],
       grad_fn=<SelectBackward0>)


In [10]:
pos_embedding_layer = nn.Embedding(4, embedding_dim)

pos_embeddings = pos_embedding_layer(torch.arange(4))
pos_embeddings.shape

torch.Size([4, 256])

In [11]:
input_embeddings = token_embeddings + pos_embeddings
input_embeddings.shape

torch.Size([8, 4, 256])